<a href="https://colab.research.google.com/github/xiashenzhen/geemap/blob/main/%E7%9B%91%E7%9D%A3%E5%88%86%E7%B1%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

监督分类测试

In [1]:
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-jason2024')


加载分类区域数据

In [ ]:
Map = geemap.Map()
point = ee.Geometry.Point([-122.4439, 37.7538])
# point = ee.Geometry.Point([117.50859, 30.65066])

image = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterBounds(point)
    .filterDate('2019-01-01', '2020-01-01')
    .sort('CLOUD_COVER')
    .first()
    .select('SR_B[1-7]')
)

image = image.multiply(0.0000275).add(-0.2).set(image.toDictionary())
vis_params = {'min': 0, 'max': 0.3, 'bands': ['SR_B5', 'SR_B4', 'SR_B3']}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")
Map

In [23]:
geemap.get_info(image)

Tree(nodes=(Node(name='Image  (7 bands)', nodes=(Node(icon='file', name='type: Image'), Node(name='bands: List…

In [24]:
image.get('DATE_ACQUIRED').getInfo()

'2019-09-24'

In [25]:
image.get('CLOUD_COVER').getInfo()

0.03

In [ ]:
nlcd = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2019')
landcover = nlcd.select('landcover').clip(image.geometry())
Map.addLayer(landcover, {}, 'NLCD Landcover')
Map

In [33]:
points = landcover.sample(
    **{
        'region': image.geometry(),
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True,
    }
)

Map.addLayer(points, {}, 'training', False)

In [28]:
print(points.size().getInfo())

3575


In [29]:
bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
label = 'landcover'
features = image.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 30}
)

In [30]:
geemap.ee_to_df(features.limit(5))

,SR_B1,SR_B2,SR_B3,SR_B4,SR_B5,SR_B6,SR_B7,landcover
0,0.024125,0.032815,0.064302,0.071975,0.293488,0.219568,0.111932,82
1,0.047060,0.059847,0.084103,0.088118,0.173835,0.137425,0.107532,24
2,0.035290,0.046318,0.077118,0.099502,0.249515,0.301050,0.188850,21
3,0.047308,0.065512,0.109513,0.146445,0.277840,0.302123,0.218550,82
4,0.058115,0.074918,0.104315,0.120375,0.197898,0.171882,0.138030,22


In [31]:
params = {

    'features': features,
    'classProperty': label,
    'inputProperties': bands,

}
classifier = ee.Classifier.smileCart(maxNodes=None).train(**params)

In [ ]:
classified = image.select(bands).classify(classifier).rename('landcover')
Map.addLayer(classified.randomVisualizer(), {}, 'Classified')
Map

In [ ]:
geemap.get_info(nlcd)

In [36]:
class_values = nlcd.get('landcover_class_values')
class_palette = nlcd.get('landcover_class_palette')
classified = classified.set({
    'landcover_class_values': class_values,
    'landcover_class_palette': class_palette
})



In [ ]:
Map.addLayer(classified, {}, 'Land cover')
Map.add_legend(title="Land cover type", builtin_legend='NLCD')
Map